In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%load_ext tensorboard
import datetime


#tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

KeyboardInterrupt: 

In [ ]:
!rm -rf ./logs/RNN #dont run this line unless you want to delete the tensorboard logs

In [ ]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [ ]:
def labels(filename):
    with open(filename, 'rb') as f:
        labels = np.load(f).ravel()
    return labels

In [ ]:
training_data_y = labels("../Labels-Train.npy")
test_y = labels("../Labels-Test.npy")

training_data_x = pickle.load(open("TS-Train.pkl", "rb"))
test_x = pickle.load(open("TS-Test.pkl", "rb"))

training_data_x = training_data_x.dropna(axis=1)
test_x = test_x.dropna(axis=1)

training_data_x = preprocessing.normalize(training_data_x, norm='max', axis=0)
test_x = preprocessing.normalize(test_x, norm='max', axis=0)

training_data_x = convert_3D(training_data_x)
test_x = convert_3D(test_x)

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [ ]:
RNNmodel = Sequential([
    SimpleRNN(32, input_shape=(training_data_x.shape[1:]), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

RNNmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/RNN/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasRNN/training.csv',separator=",", append=False)

RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = RNNmodel
model.save('keras-models/KerasRNN')

In [ ]:
%tensorboard --logdir logs/RNN/fit